In [1]:
import cv2
import numpy as np

In [2]:
'''
SPARSE OPTICAL FLOW - LUCAS KANADE METHOD.
'''
# Parameters for Shi-Tomasi corner detection algorithm.
corner_track_params = dict(maxCorners=10, qualityLevel=0.3, minDistance=7, blockSize=7)
# Parameters for Lucas Kanade object tracking algorithm.
#
# Params:
#   winSize - Window size for the object tracking. Smaller window , sensitive to noises and small
#             movements of the object over the frames. Larger frames, may not detect small movements
#             of the object and not sensitive to noise.
#
#   maxLevel - Denotes how many level on the object tracking pyramid. Higher and higher levels,
#              reduce the quality of the image by power of 2. So if the 'maxLevel=2', then the
#              image resolution multiply by 1/4 (1/2 power 2) by blurring and sub-sampling.
#              Link -> https://en.wikipedia.org/wiki/Pyramid_(image_processing)
#
#   criteria - Defines the criteria to execute Lucas Kanade algorithm.
#              cv2.TERM_CRITERIA_COUNT - Defines the iterations to search for the tracking object.
#                                        More iterations means, more exhaustive searches.
#              cv2.TERM_CRITERIA_EPS - Epsilon value defines the accuracy of the tracking. Higher
#                                      values, encourage higher accuracy, but slower tracking
#                                      throughout the frames.
#              Last two values define the number of iterations and epsilon value respectively.
#
lk_params = dict(winSize=(200, 200), maxLevel=2, criteria=(cv2.TERM_CRITERIA_COUNT | cv2.TERM_CRITERIA_EPS, 10, 0.03))

In [3]:
# Grab the image from camera.
capture = cv2.VideoCapture(0)

_, prev_frame = capture.read()

prev_gray = cv2.cvtColor(prev_frame, code=cv2.COLOR_BGR2GRAY)

# POINTS TO TRACK.
prevPts = cv2.goodFeaturesToTrack(prev_gray, mask=None, **corner_track_params)

# Mask to display the tracking points and drawing the line.
mask = np.zeros_like(prev_frame)

while capture.isOpened():

    ret, frame = capture.read()

    frame_gray = cv2.cvtColor(frame, code=cv2.COLOR_BGR2GRAY)

    # Calculate the optical flow using Pyramid Lucas Kanade.
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prevImg=prev_gray,
                                                    nextImg=frame_gray,
                                                    prevPts=prevPts,
                                                    nextPts=None,
                                                    **lk_params)

    # Filter good/matching points from the current and previous frames.
    good_new = nextPts[status==1]
    good_prev = prevPts[status==1]

    for i, (new, prev) in enumerate(zip(good_new, good_prev)):

        x_new, y_new = new.ravel()
        x_prev, y_prev = new .ravel()

        # Drawing a line between new and previous matching points on the mask.
        mask = cv2.line(mask,
                        pt1=(x_new, y_new),
                        pt2=(x_prev, y_prev),
                        color=(0, 255, 0),
                        thickness=3)

        # Drawing points on the original frame which we show.
        frame = cv2.circle(frame, center=(x_new, y_new), radius=6, color=(0, 0, 255), thickness=-1)

    img = cv2.add(frame, mask)

    cv2.imshow('Tracking', img)

    if cv2.waitKey(30) & 0xff == 27:
        break

    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1, 1, 2)
    break

capture.release()
cv2.destroyWindow('Tracking')

[[ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]]


In [13]:
'''
DENSE OPTICAL FLOW - GUNNAR FARNEBACK METHOD
'''

'\nDENSE OPTICAL FLOW - FARNEBACK METHOD\n'

In [2]:
capture = cv2.VideoCapture(0)

ret, frame1 = capture.read()

prevFrame = cv2.cvtColor(frame1, code=cv2.COLOR_BGR2GRAY)

hsv_mask = np.zeros_like(frame1)
# Set max value to Saturation channel of the mask.
hsv_mask[:, :, 1] = 255

while capture.isOpened():

    ret, frame2 = capture.read()

    nextFrame = cv2.cvtColor(frame2, code=cv2.COLOR_BGR2GRAY)
    # https://docs.opencv.org/2.4/modules/video/doc/motion_analysis_and_object_tracking.html#calcopticalflowfarneback
    flow = cv2.calcOpticalFlowFarneback(prev=prevFrame,
                                        next=nextFrame,
                                        flow=None,
                                        # By 0.5, denotes each layer of the pyramid twice smaller
                                        # than its predecessor.
                                        pyr_scale=0.5,
                                        levels=3,
                                        winsize=15,
                                        iterations=3,
                                        # Typically the value should be 5 or 7.
                                        poly_n=5,
                                        # Depends on the 'poly_n' value, If the 'poly_n=5', good
                                        # value would be 'poly_sigma=1.1', if the 'poly_n=7', good
                                        # value would be 'poly_sigma=1.5'.
                                        poly_sigma=1.2,
                                        flags=cv2.OPTFLOW_USE_INITIAL_FLOW)

    # Convert CARTESIAN 'flow' into POLAR COORDINATES.
    magnitude, angle = cv2.cartToPolar(x=flow[:, :, 0], y=flow[:, :, 1], angleInDegrees=True)

    hsv_mask[:, :, 0] = angle / 2
    hsv_mask[:, :, 2] = cv2.normalize(src=magnitude,
                                      dst=None,
                                      alpha=0,
                                      beta=255,
                                      norm_type=cv2.NORM_MINMAX)

    bgr = cv2.cvtColor(hsv_mask, code=cv2.COLOR_HSV2BGR)
    img = cv2.add(frame2, bgr)

    cv2.imshow('frame', img)

    if cv2.waitKey(20) & 0xff == 27:
        break

    prevFrame = nextFrame

capture.release()
cv2.destroyWindow('frame')